# <center> Lab 3-2: Test Send CAN
## <center> ENGR 580A2: Secure Vehicle and Industrial Networking
## <center><img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="600" />
### <center> Instructor: Dr. Jeremy Daily<br>Fall 2020

## Prerequisites
Login tip: Send a keepalive signal every 60 seconds from the connection menu of PuTTy.

This notebook must be run on a Linux machine with SocketCAN enabled.
```
sudo ip link add dev vcan1 type vcan
sudo ip link set vcan1 up
```

If you are running from PuTTy through Windows, then be sure to start the Jupyter server with the following commands:

```
jupyter notebook --ip cybertruck1.engr.colostate.edu --no-browser
```

or for the Beagle Bone Black connected by USB:
```
jupyter notebook --ip 192.168.7.2 --no-browser
```
Once these commands are successful, there will be a tokenize URL to paste into your browser. Note: this is not a secure connection, so be sure you are behind a firewall.


Be sure can-utils is installed:
```
sudo apt install can-utils
```

In [8]:
#!/usr/bin/python3
import socket
import struct

# Open a socket and bind to it from SocketCAN
sock = socket.socket(socket.PF_CAN, socket.SOCK_RAW, socket.CAN_RAW)

#Change this interface to match your desired connection
interface = "vcan1"

# Bind to the interface
sock.bind((interface,))

# To match the socketCAN data structure, the following struct format can be used:
can_frame_format = "<LB3x8s"

for i in range(25):
    # Define the CAN Arbitration ID
    can_id = 0x1FFFFFFF 
    
    #Set the extended frame format bit.
    can_id |= socket.CAN_EFF_FLAG
    
    can_data = b'eagr' 
    
    #The data length code cannot be larger than 8
    can_dlc = min(len(can_data),8)
    
    #Pack the information into 16 bytes
    can_packet = struct.pack(can_frame_format, can_id, can_dlc, can_data[:can_dlc] )
    print(can_packet)
    
    #Send out the CAN frame
    sock.send(can_packet)

b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x00\x00'
b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x00\x00'
b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x00\x00'
b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x00\x00'
b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x00\x00'
b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x00\x00'
b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x00\x00'
b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x00\x00'
b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x00\x00'
b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x00\x00'
b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x00\x00'
b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x00\x00'
b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x00\x00'
b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x00\x00'
b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x00\x00'
b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x00\x00'
b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x00\x00'
b'\xff\xff\xff\x9f\x04\x00\x00\x00eagr\x00\x00\x

## Testing and Verification
Open another terminal on the server where the interface exists. From this terminal, enter the command:
```
candump any |grep FFFFFF
```
The output should look something like this when CAN messages are sent:
```
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 00
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 01
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 02
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 03
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 04
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 05
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 06
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 07
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 08
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 09
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 0A
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 0B
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 0C
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 0D
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 0E
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 0F
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 10
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 11
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 12
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 13
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 14
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 15
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 16
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 17
  vcan1  1FFFFFFF   [7]  4A 65 72 65 6D 79 18
```


Now we have shown that we can send CAN data. Let's start looking at how we can use it for things like J1939.